<a href="https://colab.research.google.com/github/ShinAsakawa/ShinAsakawa.github.io/blob/master/notebooks/2021_0223word_associtaion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 2021 word2vec による <font color="blue">あつまれどうぶつの森</font>モデル wiith 頻度情報

工学院大学 近藤公先生との議論用です。

<div align='right'>
<a href='mailto:asakawa@ieee.org'>Shin Aasakawa</a>, all rights reserved.<br>
Date: 23/Feb/2021<br>
 MIT license
</div>

## 意味空間での操作，射影行列による語想起モデル

射影行列を用いて，意味空間を分割します。動物の名前で構成される空間をそれ以外の空間に分割します。
`動物の名前を一分間でできる限り想起してください` 課題は，この部分空間内での単語検索であると考えました。
先週お話させていただきましたソフトマックスを使ったご想起モデルでは，単語間の距離をソフトマックス関数によって
多項分布の確率密度に変換し，その確率密度関数からサンプリングとして単語想起を捉えました。
このとき，たとえば `ネコ` を手がかりとして与えた場合，`ネコ` に近い単語として `マタタビ` が想起されては困ると考えました。
そうすると，動物の名前で構成された部分空間を考えれば，`マタタビ` は想起されにくくなるだろうと考えられます。

おそらく，側頭葉に蓄えられている意味の表象に対して，前頭葉からのフィードバックが，この探索空間を限定することに相当し，
シリトリ，`カ` で始まる単語に限定する単語想起なども同様の操作で実現可能だろうと考えた次第です。

`カ` で始まる単語リスト空間への射影行列は作成済です。近日公開予定です。

このことの延長として，オノマトペ空間を考えれば，オノマトペの産出や理解 (理解の場合は，部分空間から逆に全体への逆射影) が可能になるだろうと思うのです。

基本的なアイデアの共有ですが，意味空間とその上位に位置するメタ知識，あるいはモニターとか文脈とかと言い換えても良い，という脳内機序をモデル化できるだろうと思います。

## 直交射影空間

線形代数の教科書を紐解きますと，任意のベクトルの組 $X$ の成す空間への直交射影行列は $X(X^\top X)^{-1}X^\top$ で表されます。
この行列を $P$ とすれば冪等性 $P^{n}=P$ と，その直交補空間 $C=I-P$ の冪等性 $C^n=C$ が成り立ちます。
ここで $n$ は任意の自然数です。
これらの射影行列を用いて任意の実ベクトル $y$ は $\hat{y}=Py$, $y^c=Cy=(I-P)y$ とすると $\hat{y}\perp y^{c}$ すなわち $ \hat{y}^\top y^{c} = 0$ が成り立ちます。ここで $I$ は単位行列を表すものとします。

実験計画法との関連で言えば，計画行列を $X$ とし，得られたデータを $y$ とすると，回帰行列が $X(X^\top X)X^\top$ とすれば，
$\left|\hat{y}\right|$ を自由度で除した値を $\left|y^{c}\right|$ を自由度で除した値の比が大きければ効果あり，すなわち有意差ありと判定され，
小さければ有意差なしと判定されます。

先週の動物の名前列挙モデルは，$X^\top X$ だけを使っていました。
お詫びして訂正いたします。

ここで $X$ が一つのベクトルである特殊な場合を考えれば，射影行列は次式のようになります:
$$
x(x^\top x)x^\top =\frac{xx^\top}{x^\top x},
$$
上式，分母は自らとの内積すなわちベクトルの長さに対応する量となります。$x$ が実ベクトルであれば，自身との内積は
非負 ($x^\top x\ge0$) です。
すなわち，このときの射影行列は，自分自身で構成される空間への射影ベクトルを自分自身の長さだけ大きくなるので，
その分を割って調整していることになります。


In [1]:
# -*- coding: utf-8 -*-
from six.moves.urllib import request

import numpy as np
import os
import sys
import time
import gensim

In [2]:
# word2vec の訓練済モデルを入手
!wget http://www.cis.twcu.ac.jp/~asakawa/2017jpa/2017Jul_jawiki-wakati_neologd_hid200_win20_neg20_cbow.bin.gz
#!wget http://www.cis.twcu.ac.jp/~asakawa/2017jpa/2017Jul_jawiki-wakati_neologd_hid200_win20_neg20_sgns.bin.gz
#!wget http://www.cis.twcu.ac.jp/~asakawa/2017jpa/2017Jul_jawiki-wakati_neologd_hid300_win20_neg20_sgns.bin.gz'
#!wget http://www.cis.twcu.ac.jp/~asakawa/2017jpa/2017Jul_jawiki-wakati_neologd_hid200_win20_neg20_cbow.bin.g

--2021-02-23 03:21:19--  http://www.cis.twcu.ac.jp/~asakawa/2017jpa/2017Jul_jawiki-wakati_neologd_hid200_win20_neg20_cbow.bin.gz
Resolving www.cis.twcu.ac.jp (www.cis.twcu.ac.jp)... 160.13.90.21
Connecting to www.cis.twcu.ac.jp (www.cis.twcu.ac.jp)|160.13.90.21|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.cis.twcu.ac.jp/~asakawa/2017jpa/2017Jul_jawiki-wakati_neologd_hid200_win20_neg20_cbow.bin.gz [following]
--2021-02-23 03:21:20--  https://www.cis.twcu.ac.jp/~asakawa/2017jpa/2017Jul_jawiki-wakati_neologd_hid200_win20_neg20_cbow.bin.gz
Connecting to www.cis.twcu.ac.jp (www.cis.twcu.ac.jp)|160.13.90.21|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1369049082 (1.3G) [application/x-gzip]
Saving to: ‘2017Jul_jawiki-wakati_neologd_hid200_win20_neg20_cbow.bin.gz’

2017Jul_jawiki-waka 100%[===================>]   1.27G  13.2MB/s    in 1m 46s  

2021-02-23 03:23:06 (12.3 MB/s) - ‘2017Jul_jawiki-wakati_neologd_hid200_win20

In [3]:
word2vec_file='2017Jul_jawiki-wakati_neologd_hid200_win20_neg20_cbow.bin.gz'
model = gensim.models.KeyedVectors.load_word2vec_format(word2vec_file, 
                                                        encoding='utf-8', 
                                                        unicode_errors='replace', 
                                                        binary=True)
w2v = model

In [ ]:
def free_word_assoc(word, topn=64, beta=0.03, n_seq=10, seed=int(time.time()), verbose=False, w2v=w2v):
    """
    自由連想課題のシミュレーション: 単語 word を刺激語として逐次自由連想を繰り返す
    パラメータ:
    word: str
        タネとなる単語
    n_seq: int
        連想をする回数
    topn: int
        想起する単語の最大数，上位 topn 個の単語までが考慮される
    beta: float
        温度パラメータ，低いと決定論的に振る舞い，高いとその都度振る舞いが異なる
    seed: int
        乱数の種 指定しなければ現在の時刻を用いる。指定すると再現性が保証される
    verbose: bool
        冗長な出力を行う
    w2v: class(word2vec)
        word2vec の本体
    """
    np.random.seed(seed)
    ret = []
    for i in range(n_seq):
        # 最近隣語を topn 語だけ得る
        vec = w2v.similar_by_word(word, topn=topn)
        
        # 0 は最近隣語であるので, その次の 1 以降をすべて用いる
        sims = np.array([x[1] for x in vec][1:])
        
        # 単語を格納
        names = [x[0] for x in vec][1:]
        
        # ソフトマックス関数を使って確率に変換
        prob = np.exp(sims/beta) / np.exp(sims/beta).sum()
        
        # 確率に基づいてランダムサンプリング
        word = np.random.choice(names,p=prob)
        
        # サンプリングして得た結果を格納
        ret.append(word)
        if verbose:
            print(word, end="")
            print(' :', names.index(word)+1, names, prob)
    return ret

# このセルで定義した自由連想モデルを試してみる。
free_word_assoc('パンダ', n_seq=5, beta=0.02, verbose=False)

In [ ]:
# ここから先がモデル，生き物に関係しそうな単語を使って動物リストを作成
words = ['生き物', '動物', '生物', '生きもの', '哺乳類', '爬虫類', '鳥類', '魚類', '両生類']

w_list = []
for word in words:
    
    # topn=256 で最近接語 上位 256 語を選ぶ
    for x in w2v.most_similar(word, topn=256):
        w_list.append(x[0])
    print(word, end=' ') #, [x[0] for x in w2v.most_similar(word, topn=512)])

print('\n', sorted(set(w_list)))

In [6]:
# 動物を上の情報から動物の名前らしき項目だけ，勝手に選ぶ
#animals = ['イヌ', 'インコ', 'ウォッチング', 'ウサギ', 'カワウソ', 
#           'キジ', 'クジラ', 'サメ', 'スズメ', 'ティラノサウルス', 
#           'トカゲ', 'ネズミ', 'バクテリア', 'ヒト', 'フクロウ', 
#           'プランクトン', 'ヘビ', 'ムツゴロウ', 'ヤンバルクイナ', 'ワニ']
animals = ['おたまじゃくし', 'アオウミガメ', 'アオサギ', 'アオダイショウ', 'アカアシシギ', 
           'アカエイ', 'アカオネッタイチョウ', 'アカガエル', 'アカギツネ', 'アカゲザル', 
           'アカゲラ', 'アカシカ', 'アカテガニ', 'アカネズミ', 'アカボウクジラ', 'アカントステガ', 
           'アクロカントサウルス', 'アザラシ', 'アシカ', 'アジサシ', 'アナウサギ', 
           'アナグマ', 'アノマロカリス', 'アフリカゾウ', 'アフリカツメガエル', 'アブラツノザメ', 
           'アホウドリ', 'アマガエル', 'アマツバメ', 'アマモ', 'アミメニシキヘビ', 'アメフラシ', 
           'アメリカバイソン', 'アメリカワシミミズク', 'アメンボ', 'アメーバ', 'アユ', 
           'アライグマ', 'アリクイ', 'アリゲーターガー', 'アリジゴク', 'アルマジロ', 'アロサウルス', 
           'アロワナ', 'アンキロサウルス', 'アンコウ', 'アンテオサウルス', 'アンドリューサルクス', 
           'アンモナイト', 'アーケオプテリクス', 'アーケロン', 'アースロプレウラ', 'イエネコ', 
           'イクチオサウルス', 'イクチオステガ', 'イグアナ', 'イグアノドン', 'イサキ', 'イシガイ', 
           'イシサンゴ', 'イソギンチャク', 'イタセンパラ', 'イタチザメ', 'イッカク', 'イヌ', 
           'イヌワシ', 'イノシシ', 'イボニシ', 'イモムシ', 'イモリ', 'イリエワニ', 
           'イリオモテヤマネコ', 'イルカ', 'イロワケイルカ', 'イワシ', 'インコ', 'インドガビアル', 
           'インドクジャク', 'インドハッカ', 'ウイルス', 'ウグイ', 'ウサギ', 'ウシ', 
           'ウシガエル', 'ウズラ', 'ウツボ', 'ウナギ', 'ウバザメ', 'ウマ', 'ウミウシ', 
           'ウミガメ', 'ウミクワガタ', 'ウミサソリ', 'ウミタナゴ', 'ウミツバメ', 'ウミニナ', 
           'ウミヘビ', 'ウミユリ', 'ウンピョウ', 'エイリアン', 'エウステノプテロン', 'エゾヒグマ', 
           'エダフォサウルス', 'エチゼンクラゲ', 'エナガ', 'エピオルニス', 'エボシガイ', 'エミュー', 
           'オウムガイ', 'オオウナギ', 'オオウミガラス', 'オオカミ', 'オオコウモリ', 'オオサンショウウオ', 
           'オオセ', 'オオタカ', 'オオツノシカ', 'オオトカゲ', 'オオハシ', 'オオハシウミガラス', 'オオバン', 
           'オオフラミンゴ', 'オオマリコケムシ', 'オオメジロザメ', 'オオヤマネコ', 'オキアミ', 'オジロジカ', 
           'オジロワシ', 'オタマジャクシ', 'オットセイ', 'オナガガモ', 'オニアンコウ', 'オニオオハシ', 
           'オニヒトデ', 'オポッサム', 'オマキザル', 'オランウータン', 'オルガネラ', 'オヴィラプトロサウルス', 
           'カイウサギ', 'カイギュウ', 'カイメン', 'カエル', 'カギムシ', 'カサゴ', 'カジキ', 'カタクチイワシ', 
           'カタツムリ', 'カダヤシ', 'カッコウ', 'カツオクジラ', 'カツオドリ', 'カツオノエボシ', 'カナダガン', 
           'カニ', 'カニクイアザラシ', 'カニクイザル', 'カニダマシ', 'カバ', 'カピバラ', 'カブトガニ', 
           'カブトムシ', 'カマキリ', 'カメ', 'カメムシ', 'カモ', 'カモノハシ', 'カヤネズミ', 'カライワシ', 
           'カラシン', 'カラス', 'カルカロドントサウルス', 'カルガモ', 'カレイ', 'カワウソ', 'カワガラス', 
           'カワゲラ', 'カワスズメ', 'カワセミ', 'カワニナ', 'カワハギ', 'カワラヒワ', 'カンガルー', 
           'ガチョウ', 'キジ', 'キジバト', 'キセルガイ', 'キタキツネ', 'キタリス', 'キツツキ', 'キツネ', 
           'キツネザル', 'キバタン', 'キュウリウオ', 'キンクロハジロ', 'キンチャクダイ', 'ギガノトサウルス', 
           'ギフチョウ', 'ギンダラ', 'ギンブナ', 'クサガメ', 'クサフグ', 'クジラ', 'クニマス', 'クマ', 
           'クマネズミ', 'クマノミ', 'クモ', 'クモザル', 'クラゲ', 'クラーケン', 'クリオネ', 'クリダグナサス', 
           'クリーパー', 'クルマエビ', 'クロツヤムシ', 'クロトキ', 'クロナガアリ', 'クロマグロ', 'クワガタムシ', 
           'グッピー', 'グリプトドン', 'グリーンイグアナ', 'グンカンドリ', 'ケツァルコアトルス', 'ケンミジンコ', 
           'ケンムン', 'ケープハイラックス', 'ケープペンギン', 'ゲッチョ', 'ゲラダヒヒ', 'ゲンゴロウ', 
           'コアジサシ', 'コイ', 'コウイカ', 'コウノトリ', 'コウモリ', 'コオロギ', 'コクチョウ', 
           'コゲラ', 'コサギ', 'コジュケイ', 'コノドント', 'コビトカバ', 'コブハクチョウ', 'コモドオオトカゲ', 
           'コンゴウインコ', 'コンプソグナトゥス', 'ゴイサギ', 'ゴカイ', 'ゴキブリ', 'ゴマフアザラシ', 
           'ゴルゴサウルス', 'ゴルゴノプス', 'ゴンズイ', 'ゴンドウクジラ', 'サイチョウ', 'サカマキガイ', 
           'サケ', 'サケイ', 'サソリ', 'サメ', 'サル', 'サワガニ', 'サンゴ', 'サンショウウオ', 'ザトウクジラ', 
           'ザリガニ', 'シアノバクテリア', 'シカ', 'シカクナマコ', 'シギダチョウ', 'シクリッド', 'シジュウカラ', 
           'シノカリオプテリクス', 'シノサウロプテリクス', 'シファクティヌス', 'シマウマ', 'シマフクロウ', 
           'シャチ', 'シュモクザメ', 'ショウジョウバエ', 'シラウオ', 'シラミ', 'シロアリ', 'シロイルカ', 
           'シロウオ', 'シロサイ', 'シロナガスクジラ', 'シーサーペント', 'シーラカンス', 'ジャガーネコ', 
           'ジュゴン', 'ジリス', 'ジンベエザメ', 'スイギュウ', 'スカフォグナトゥス', 'スクミリンゴガイ', 
           'スジイルカ', 'スジエビ', 'スズガモ', 'スズメ', 'スズメガ', 'スズメダイ', 'スティラコサウルス', 
           'ステゴサウルス', 'ストロマトライト', 'スナギンチャク', 'スナネズミ', 'スナメリ', 'スネール', 
           'スピノサウルス', 'スミロドン', 'スローロリス', 'セイウチ', 'セイヨウミツバチ', 'セグロカモメ', 
           'セミクジラ', 'センザンコウ', 'ソコダラ', 'ゾウ', 'ゾウアザラシ', 'ゾウリムシ', 'タイコウチ', 
           'タガメ', 'タキタロウ', 'タコ', 'タスマニアデビル', 'タツノオトシゴ', 'タナゴ', 'タニシ', 
           'タヌキ', 'タンチョウ', 'ダイオウイカ', 'ダイサギ', 'ダチョウ', 'ダツ', 'ダマジカ', 'ダルマザメ', 
           'ダンクルオステウス', 'ダンゴムシ', 'チューブワーム', 'チョウ', 'チョウザメ', 'チョウチョウウオ', 
           'チョウチンアンコウ', 'チリモン', 'チンチラ', 'チンパンジー', 'ツェツェバエ', 'ツキノワグマ', 
           'ツチガエル', 'ツチスドリ', 'ツチノコ', 'ツチブタ', 'ツノザメ', 'ティラノサウルス', 'ティロサウルス', 
           'テッポウエビ', 'テナガエビ', 'テナガザル', 'テノントサウルス', 'テミンク', 'テンジクネズミ', 
           'テントウムシ', 'ディアデクテス', 'ディキノドン', 'ディノケファルス', 'ディモルフォドン', 
           'デイノテリウム', 'デイノニクス', 'デジモン', 'デスモスチルス', 'デルフチア・アシドボランス', 
           'トカゲ', 'トガリネズミ', 'トゲウオ', 'トビケラ', 'トビハゼ', 'トリガイ', 'トリケラトプス', 
           'トルヴォサウルス', 'トロオドン', 'ドクトカゲ', 'ドジョウ', 'ドブネズミ', 'ドロマエオサウルス', 
           'ドードー', 'ナガスクジラ', 'ナキウサギ', 'ナマケモノ', 'ナマコ', 'ナマズ', 'ナメクジ', 
           'ニゴイ', 'ニザダイ', 'ニシツノメドリ', 'ニシローランドゴリラ', 'ニジマス', 'ニタリクジラ', 
           'ニホンアカガエル', 'ニホンイシガメ', 'ニホンウナギ', 'ニホンカナヘビ', 'ニホンカワウソ', 
           'ニホンザリガニ', 'ニホンザル', 'ニホントカゲ', 'ニホンヤモリ', 'ニューギニアヤリガタリクウズムシ', 
           'ニワトリ', 'ヌタウナギ', 'ヌマエビ', 'ネコ', 'ネズッポ', 'ネズミ', 'ネズミイルカ', 'ネズミザメ', 
           'ネッシー', 'ノウサギ', 'ノコギリエイ', 'ノネズミ', 'ノーザンパイク', 'ハイイロガン', 'ハイギョ', 
           'ハエ', 'ハオリムシ', 'ハキリアリ', 'ハクジラ', 'ハクビシン', 'ハゲワシ', 'ハコフグ', 'ハシビロガモ', 
           'ハシビロコウ', 'ハジラミ', 'ハゼ', 'ハタネズミ', 'ハダカイワシ', 'ハダカデバネズミ', 'ハチ', 
           'ハチドリ', 'ハツカネズミ', 'ハト目', 'ハナムグリ', 'ハリネズミ', 'ハリモグラ', 'ハーテビースト', 
           'バイカルアザラシ', 'バクテリア', 'バッタ', 'バンドウイルカ', 'パキケトゥス', 'パキケファロサウルス', 
           'パレイアサウルス', 'ヒキガエル', 'ヒクイドリ', 'ヒグマ', 'ヒゲクジラ', 'ヒツジ', 'ヒト', 'ヒトデ', 
           'ヒドリガモ', 'ヒナモロコ', 'ヒヒ', 'ヒプシロフォドン', 'ヒヨケザル', 'ヒヨドリ', 'ヒラメ',
           'ピコプランクトン', 'ピラニア', 'ピラルクー', 'ピーシーズ', 'フェイゾン', 'フエガラス', 
           'フォルスラコス', 'フクロウ', 'フクロウオウム', 'フクロオオカミ', 'フクロネコ', 'フクロネズミ', 
           'フクロモモンガ', 'フグ', 'フジツボ', 'フデイシ', 'フルマカモメ', 'ブタ', 'ブチハイエナ', 
           'ブッポウソウ', 'ブラキオサウルス', 'プテラノドン', 'プテロダクティルス', 'プラナリア', 
           'プランクトン', 'プレシオサウルス', 'プレソドン', 'プレーリードッグ', 'プロガノケリス', 
           'プロトプラスト', 'ヘコアユ', 'ヘスペロルニス', 'ヘビ', 'ヘビクイワシ', 'ベニイロフラミンゴ', 
           'ベニザケ', 'ベンガルトラ', 'ベントス', 'ペッカリー', 'ペット', 'ペラゴルニス', 'ペリカン', 
           'ペリット', 'ペンギン', 'ホウボウ', 'ホオダレムクドリ', 'ホシハジロ', 'ホシムクドリ', 'ホタル', 
           'ホタルイカ', 'ホッキョクギツネ', 'ホッキョクグマ', 'ホホジロザメ', 'ホラアナグマ', 'ホライモリ', 
           'ホワイトバス', 'ホンソメワケベラ', 'ボノボ', 'ボラジョ', 'ポリプテルス', 'マイワシ', 'マガモ', 
           'マジュンガサウルス', 'マダイ', 'マダコ', 'マダニ', 'マッコウクジラ', 'マナヅル', 'マナティー', 
           'マハゼ', 'マプサウルス', 'マルハナバチ', 'マレーグマ', 'マンジュウダイ', 'マンモス', 
           'マーモセット', 'マーモット', 'マーレーコッド', 'ミクロラプトル', 'ミサゴ', 'ミシシッピアカミミガメ', 
           'ミジンコ', 'ミズガメ', 'ミズナギドリ', 'ミツバチ', 'ミトコンドリア', 'ミドリムシ', 
           'ミナミゾウアザラシ', 'ミミズ', 'ミミズク', 'ミュールジカ', 'ミーアキャット', 'ムカシトカゲ', 
           'ムカシトンボ', 'ムカデ', 'ムクドリ', 'ムササビ', 'ムツゴロウ', 'ムラサキウニ', 'メガテリウム', 
           'メガロドン', 'メジロザメ', 'メダカ', 'メトリオリンクス', 'メトロイド', 'メバル', 'モエビ', 
           'モクズガニ', 'モグラ', 'モササウルス', 'モツゴ', 'モノアラガイ', 'モモイロペリカン', 
           'モルモット', 'モンガラカワハギ', 'モーグリ', 'ヤギ', 'ヤシガニ', 'ヤツメウナギ', 'ヤドカリ', 
           'ヤノマミ', 'ヤブカ', 'ヤマアカガエル', 'ヤマアラシ', 'ヤマカガシ', 'ヤマシギ', 'ヤマトシジミ', 
           'ヤモリ', 'ヤンバルクイナ', 'ユスリカ', 'ユリカモメ', 'ヨコエソ', 'ヨコエビ', 'ヨコバイ', 
           'ヨシノボリ', 'ヨロイザメ', 'ヨーロッパヒメウ', 'ライチョウ', 'ラッコ', 'ランフォリンクス', 
           'リオプレウロドン', 'リャマ', 'ルリオーストラリアムシクイ', 'ルリビタキ', 'レイクトラウト', 
           'レイヨウ', 'レッサーパンダ', 'レプトケファルス', 'ロボク', 'ワシミミズク', 'ワタリガニ', 
           'ワニ', 'ワニトカゲギス', 'ワモンアザラシ', 'ワライカワセミ', 'ワラジムシ', 'ヴェロキラプトル', 
           '回虫', '始祖鳥', '山羊', '巻き貝', '巻貝', '栗鼠', '毛虫', '犬', '猫', '猿', '硬骨魚', 
           '蛇', '蛙', '蛾', '蜘蛛', '蠕虫', '豚', '象', '貝', '野ネズミ', '金魚', '鯨']

# 勝手に選んだ animals の リストからどうぶつの射影行列 P を作成
P = np.zeros((len(animals), w2v['イヌ'].shape[0]))
for i, animal in enumerate(animals):
    P[i] = np.copy(w2v[animal])
wProj = np.dot(P.T, P)    

P_PT = np.dot(P, P.T)
Pinv = np.linalg.inv(P_PT)
Proj = np.dot(P.T, np.dot(Pinv, P))

# 直交射影行列の作成
I = np.eye(len(Proj))
C = I - Proj


In [7]:
# 試しに イヌ ベクトルを 射影行列に作用させ どうぶつの森空間ベクトル  を作成
word = '知能'
x = np.array(w2v[word])

print('# 直交射影空間上のベクトル: ', end='')
print([x[0] for x in w2v.similar_by_vector(np.dot(Proj,x), topn=5)])

print('# 直交補空間上のベクトル: ', end='')
print([x[0] for x in w2v.similar_by_vector(np.dot(C,x), topn=5)])

print('# 射影空間上でのベクトル: ', end='')
print([x[0] for x in w2v.similar_by_vector(np.dot(wProj, x), topn=5)])

# 直交射影空間上のベクトル: ['1億分の8', 'ウナム・サンクタム', '角橋', '無名都市', 'ヌガラ']
# 直交補空間上のベクトル: ['nameEi', 'ヒイアカ', '92時間', '延命措置', '死に別れる']
# 射影空間上でのベクトル: ['ヘビ', 'ワニ', 'アリクイ', 'カワウソ', 'ザリガニ']


In [8]:
# 項目名
word = 'イヌ'
x = np.array(w2v[word])
entries = [x[0] for x in w2v.similar_by_vector(np.dot(wProj,x), topn=32)][1:]

# 各項目の類似度
Sim_entries = np.array([x[1] for x in w2v.similar_by_vector(x, topn=32)])[1:]

# ソフトマックスを使って確率密度に変換する
beta = 0.05
Sim_max = np.exp(Sim_entries/beta).sum()
P_sim = np.exp(Sim_entries/beta)/Sim_max

# 上の確率密度を使ってランダムサンプリングしてみる
word = np.random.choice(entries,p=P_sim)

# 結果の印字
print(word)

ワニ


In [16]:
# 上記を使ってシミュレーションモデルを作成

class animal_forest():
    def __init__(self, w2v=w2v, animals=False):
        
        self.w2v = w2v
        if not animals:
            self.animals = ['おたまじゃくし', 'アオウミガメ', 'アオサギ', 'アオダイショウ', 'アカアシシギ', 
                            'アカエイ', 'アカオネッタイチョウ', 'アカガエル', 'アカギツネ', 'アカゲザル', 
                            'アカゲラ', 'アカシカ', 'アカテガニ', 'アカネズミ', 'アカボウクジラ', 'アカントステガ', 
                            'アクロカントサウルス', 'アザラシ', 'アシカ', 'アジサシ', 'アナウサギ', 
                            'アナグマ', 'アノマロカリス', 'アフリカゾウ', 'アフリカツメガエル', 'アブラツノザメ', 
                            'アホウドリ', 'アマガエル', 'アマツバメ', 'アマモ', 'アミメニシキヘビ', 'アメフラシ', 
                            'アメリカバイソン', 'アメリカワシミミズク', 'アメンボ', 'アメーバ', 'アユ', 
                            'アライグマ', 'アリクイ', 'アリゲーターガー', 'アリジゴク', 'アルマジロ', 'アロサウルス', 
                            'アロワナ', 'アンキロサウルス', 'アンコウ', 'アンテオサウルス', 'アンドリューサルクス', 
                            'アンモナイト', 'アーケオプテリクス', 'アーケロン', 'アースロプレウラ', 'イエネコ', 
                            'イクチオサウルス', 'イクチオステガ', 'イグアナ', 'イグアノドン', 'イサキ', 'イシガイ', 
                            'イシサンゴ', 'イソギンチャク', 'イタセンパラ', 'イタチザメ', 'イッカク', 'イヌ', 
                            'イヌワシ', 'イノシシ', 'イボニシ', 'イモムシ', 'イモリ', 'イリエワニ', 
                            'イリオモテヤマネコ', 'イルカ', 'イロワケイルカ', 'イワシ', 'インコ', 'インドガビアル', 
                            'インドクジャク', 'インドハッカ', 'ウイルス', 'ウグイ', 'ウサギ', 'ウシ', 
                            'ウシガエル', 'ウズラ', 'ウツボ', 'ウナギ', 'ウバザメ', 'ウマ', 'ウミウシ', 
                            'ウミガメ', 'ウミクワガタ', 'ウミサソリ', 'ウミタナゴ', 'ウミツバメ', 'ウミニナ', 
                            'ウミヘビ', 'ウミユリ', 'ウンピョウ', 'エイリアン', 'エウステノプテロン', 'エゾヒグマ', 
                            'エダフォサウルス', 'エチゼンクラゲ', 'エナガ', 'エピオルニス', 'エボシガイ', 'エミュー', 
                            'オウムガイ', 'オオウナギ', 'オオウミガラス', 'オオカミ', 'オオコウモリ', 'オオサンショウウオ', 
                            'オオセ', 'オオタカ', 'オオツノシカ', 'オオトカゲ', 'オオハシ', 'オオハシウミガラス', 'オオバン', 
                            'オオフラミンゴ', 'オオマリコケムシ', 'オオメジロザメ', 'オオヤマネコ', 'オキアミ', 'オジロジカ', 
                            'オジロワシ', 'オタマジャクシ', 'オットセイ', 'オナガガモ', 'オニアンコウ', 'オニオオハシ', 
                            'オニヒトデ', 'オポッサム', 'オマキザル', 'オランウータン', 'オルガネラ', 'オヴィラプトロサウルス', 
                            'カイウサギ', 'カイギュウ', 'カイメン', 'カエル', 'カギムシ', 'カサゴ', 'カジキ', 'カタクチイワシ', 
                            'カタツムリ', 'カダヤシ', 'カッコウ', 'カツオクジラ', 'カツオドリ', 'カツオノエボシ', 'カナダガン', 
                            'カニ', 'カニクイアザラシ', 'カニクイザル', 'カニダマシ', 'カバ', 'カピバラ', 'カブトガニ', 
                            'カブトムシ', 'カマキリ', 'カメ', 'カメムシ', 'カモ', 'カモノハシ', 'カヤネズミ', 'カライワシ', 
                            'カラシン', 'カラス', 'カルカロドントサウルス', 'カルガモ', 'カレイ', 'カワウソ', 'カワガラス', 
                            'カワゲラ', 'カワスズメ', 'カワセミ', 'カワニナ', 'カワハギ', 'カワラヒワ', 'カンガルー', 
                            'ガチョウ', 'キジ', 'キジバト', 'キセルガイ', 'キタキツネ', 'キタリス', 'キツツキ', 'キツネ', 
                            'キツネザル', 'キバタン', 'キュウリウオ', 'キンクロハジロ', 'キンチャクダイ', 'ギガノトサウルス', 
                            'ギフチョウ', 'ギンダラ', 'ギンブナ', 'クサガメ', 'クサフグ', 'クジラ', 'クニマス', 'クマ', 
                            'クマネズミ', 'クマノミ', 'クモ', 'クモザル', 'クラゲ', 'クラーケン', 'クリオネ', 'クリダグナサス', 
                            'クリーパー', 'クルマエビ', 'クロツヤムシ', 'クロトキ', 'クロナガアリ', 'クロマグロ', 'クワガタムシ', 
                            'グッピー', 'グリプトドン', 'グリーンイグアナ', 'グンカンドリ', 'ケツァルコアトルス', 'ケンミジンコ', 
                            'ケンムン', 'ケープハイラックス', 'ケープペンギン', 'ゲッチョ', 'ゲラダヒヒ', 'ゲンゴロウ', 
                            'コアジサシ', 'コイ', 'コウイカ', 'コウノトリ', 'コウモリ', 'コオロギ', 'コクチョウ', 
                            'コゲラ', 'コサギ', 'コジュケイ', 'コノドント', 'コビトカバ', 'コブハクチョウ', 'コモドオオトカゲ', 
                            'コンゴウインコ', 'コンプソグナトゥス', 'ゴイサギ', 'ゴカイ', 'ゴキブリ', 'ゴマフアザラシ', 
                            'ゴルゴサウルス', 'ゴルゴノプス', 'ゴンズイ', 'ゴンドウクジラ', 'サイチョウ', 'サカマキガイ', 
                            'サケ', 'サケイ', 'サソリ', 'サメ', 'サル', 'サワガニ', 'サンゴ', 'サンショウウオ', 'ザトウクジラ', 
                            'ザリガニ', 'シアノバクテリア', 'シカ', 'シカクナマコ', 'シギダチョウ', 'シクリッド', 'シジュウカラ', 
                            'シノカリオプテリクス', 'シノサウロプテリクス', 'シファクティヌス', 'シマウマ', 'シマフクロウ', 
                            'シャチ', 'シュモクザメ', 'ショウジョウバエ', 'シラウオ', 'シラミ', 'シロアリ', 'シロイルカ', 
                            'シロウオ', 'シロサイ', 'シロナガスクジラ', 'シーサーペント', 'シーラカンス', 'ジャガーネコ', 
                            'ジュゴン', 'ジリス', 'ジンベエザメ', 'スイギュウ', 'スカフォグナトゥス', 'スクミリンゴガイ', 
                            'スジイルカ', 'スジエビ', 'スズガモ', 'スズメ', 'スズメガ', 'スズメダイ', 'スティラコサウルス', 
                            'ステゴサウルス', 'ストロマトライト', 'スナギンチャク', 'スナネズミ', 'スナメリ', 'スネール', 
                            'スピノサウルス', 'スミロドン', 'スローロリス', 'セイウチ', 'セイヨウミツバチ', 'セグロカモメ', 
                            'セミクジラ', 'センザンコウ', 'ソコダラ', 'ゾウ', 'ゾウアザラシ', 'ゾウリムシ', 'タイコウチ', 
                            'タガメ', 'タキタロウ', 'タコ', 'タスマニアデビル', 'タツノオトシゴ', 'タナゴ', 'タニシ', 
                            'タヌキ', 'タンチョウ', 'ダイオウイカ', 'ダイサギ', 'ダチョウ', 'ダツ', 'ダマジカ', 'ダルマザメ', 
                            'ダンクルオステウス', 'ダンゴムシ', 'チューブワーム', 'チョウ', 'チョウザメ', 'チョウチョウウオ', 
                            'チョウチンアンコウ', 'チリモン', 'チンチラ', 'チンパンジー', 'ツェツェバエ', 'ツキノワグマ', 
                            'ツチガエル', 'ツチスドリ', 'ツチノコ', 'ツチブタ', 'ツノザメ', 'ティラノサウルス', 'ティロサウルス', 
                            'テッポウエビ', 'テナガエビ', 'テナガザル', 'テノントサウルス', 'テミンク', 'テンジクネズミ', 
                            'テントウムシ', 'ディアデクテス', 'ディキノドン', 'ディノケファルス', 'ディモルフォドン', 
                            'デイノテリウム', 'デイノニクス', 'デジモン', 'デスモスチルス', 'デルフチア・アシドボランス', 
                            'トカゲ', 'トガリネズミ', 'トゲウオ', 'トビケラ', 'トビハゼ', 'トリガイ', 'トリケラトプス', 
                            'トルヴォサウルス', 'トロオドン', 'ドクトカゲ', 'ドジョウ', 'ドブネズミ', 'ドロマエオサウルス', 
                            'ドードー', 'ナガスクジラ', 'ナキウサギ', 'ナマケモノ', 'ナマコ', 'ナマズ', 'ナメクジ', 
                            'ニゴイ', 'ニザダイ', 'ニシツノメドリ', 'ニシローランドゴリラ', 'ニジマス', 'ニタリクジラ', 
                            'ニホンアカガエル', 'ニホンイシガメ', 'ニホンウナギ', 'ニホンカナヘビ', 'ニホンカワウソ', 
                            'ニホンザリガニ', 'ニホンザル', 'ニホントカゲ', 'ニホンヤモリ', 'ニューギニアヤリガタリクウズムシ', 
                            'ニワトリ', 'ヌタウナギ', 'ヌマエビ', 'ネコ', 'ネズッポ', 'ネズミ', 'ネズミイルカ', 'ネズミザメ', 
                            'ネッシー', 'ノウサギ', 'ノコギリエイ', 'ノネズミ', 'ノーザンパイク', 'ハイイロガン', 'ハイギョ', 
                            'ハエ', 'ハオリムシ', 'ハキリアリ', 'ハクジラ', 'ハクビシン', 'ハゲワシ', 'ハコフグ', 'ハシビロガモ', 
                            'ハシビロコウ', 'ハジラミ', 'ハゼ', 'ハタネズミ', 'ハダカイワシ', 'ハダカデバネズミ', 'ハチ', 
                            'ハチドリ', 'ハツカネズミ', 'ハト目', 'ハナムグリ', 'ハリネズミ', 'ハリモグラ', 'ハーテビースト', 
                            'バイカルアザラシ', 'バクテリア', 'バッタ', 'バンドウイルカ', 'パキケトゥス', 'パキケファロサウルス', 
                            'パレイアサウルス', 'ヒキガエル', 'ヒクイドリ', 'ヒグマ', 'ヒゲクジラ', 'ヒツジ', 'ヒト', 'ヒトデ', 
                            'ヒドリガモ', 'ヒナモロコ', 'ヒヒ', 'ヒプシロフォドン', 'ヒヨケザル', 'ヒヨドリ', 'ヒラメ',
                            'ピコプランクトン', 'ピラニア', 'ピラルクー', 'ピーシーズ', 'フェイゾン', 'フエガラス', 
                            'フォルスラコス', 'フクロウ', 'フクロウオウム', 'フクロオオカミ', 'フクロネコ', 'フクロネズミ', 
                            'フクロモモンガ', 'フグ', 'フジツボ', 'フデイシ', 'フルマカモメ', 'ブタ', 'ブチハイエナ', 
                            'ブッポウソウ', 'ブラキオサウルス', 'プテラノドン', 'プテロダクティルス', 'プラナリア', 
                            'プランクトン', 'プレシオサウルス', 'プレソドン', 'プレーリードッグ', 'プロガノケリス', 
                            'プロトプラスト', 'ヘコアユ', 'ヘスペロルニス', 'ヘビ', 'ヘビクイワシ', 'ベニイロフラミンゴ', 
                            'ベニザケ', 'ベンガルトラ', 'ベントス', 'ペッカリー', 'ペット', 'ペラゴルニス', 'ペリカン', 
                            'ペリット', 'ペンギン', 'ホウボウ', 'ホオダレムクドリ', 'ホシハジロ', 'ホシムクドリ', 'ホタル', 
                            'ホタルイカ', 'ホッキョクギツネ', 'ホッキョクグマ', 'ホホジロザメ', 'ホラアナグマ', 'ホライモリ', 
                            'ホワイトバス', 'ホンソメワケベラ', 'ボノボ', 'ボラジョ', 'ポリプテルス', 'マイワシ', 'マガモ', 
                            'マジュンガサウルス', 'マダイ', 'マダコ', 'マダニ', 'マッコウクジラ', 'マナヅル', 'マナティー', 
                            'マハゼ', 'マプサウルス', 'マルハナバチ', 'マレーグマ', 'マンジュウダイ', 'マンモス', 
                            'マーモセット', 'マーモット', 'マーレーコッド', 'ミクロラプトル', 'ミサゴ', 'ミシシッピアカミミガメ', 
                            'ミジンコ', 'ミズガメ', 'ミズナギドリ', 'ミツバチ', 'ミトコンドリア', 'ミドリムシ', 
                            'ミナミゾウアザラシ', 'ミミズ', 'ミミズク', 'ミュールジカ', 'ミーアキャット', 'ムカシトカゲ', 
                            'ムカシトンボ', 'ムカデ', 'ムクドリ', 'ムササビ', 'ムツゴロウ', 'ムラサキウニ', 'メガテリウム', 
                            'メガロドン', 'メジロザメ', 'メダカ', 'メトリオリンクス', 'メトロイド', 'メバル', 'モエビ', 
                            'モクズガニ', 'モグラ', 'モササウルス', 'モツゴ', 'モノアラガイ', 'モモイロペリカン', 
                            'モルモット', 'モンガラカワハギ', 'モーグリ', 'ヤギ', 'ヤシガニ', 'ヤツメウナギ', 'ヤドカリ', 
                            'ヤノマミ', 'ヤブカ', 'ヤマアカガエル', 'ヤマアラシ', 'ヤマカガシ', 'ヤマシギ', 'ヤマトシジミ', 
                            'ヤモリ', 'ヤンバルクイナ', 'ユスリカ', 'ユリカモメ', 'ヨコエソ', 'ヨコエビ', 'ヨコバイ', 
                            'ヨシノボリ', 'ヨロイザメ', 'ヨーロッパヒメウ', 'ライチョウ', 'ラッコ', 'ランフォリンクス', 
                            'リオプレウロドン', 'リャマ', 'ルリオーストラリアムシクイ', 'ルリビタキ', 'レイクトラウト', 
                            'レイヨウ', 'レッサーパンダ', 'レプトケファルス', 'ロボク', 'ワシミミズク', 'ワタリガニ', 
                            'ワニ', 'ワニトカゲギス', 'ワモンアザラシ', 'ワライカワセミ', 'ワラジムシ', 'ヴェロキラプトル', 
                            '回虫', '始祖鳥', '山羊', '巻き貝', '巻貝', '栗鼠', '毛虫', '犬', '猫', '猿', '硬骨魚', 
                            '蛇', '蛙', '蛾', '蜘蛛', '蠕虫', '豚', '象', '貝', '野ネズミ', '金魚', '鯨']
            # animals の リストからどうぶつの射影行列 P を作成
            P = np.zeros((len(self.animals), self.w2v['イヌ'].shape[0]))
            for i, animal in enumerate(self.animals):
                P[i] = np.copy(self.w2v[animal])
            P_PT = np.dot(P, P.T)
            Pinv = np.linalg.inv(P_PT)
            self.Proj = np.dot(P.T, np.dot(Pinv, P))
            self.wProj = np.dot(P.T, P)
            self.C = np.eye(len(self.Proj)) - self.Proj

            
    def word_assoc(self, word, Proj=None, n_seq=5, topn=32, beta=0.03, seed=int(time.time()), verbose=False, w2v=w2v):
        """
        自由連想課題のシミュレーション: 単語 word を刺激語として逐次自由連想を繰り返す
        パラメータ:
            word: str
                刺激語 
            Proj: np.array
                動物名空間への射影行列
            n_seq: int
                自由連想を続ける回数
            topn: int
                想起する単語の最大数，上位 topn 個の単語までが考慮される
            beta: float
                温度パラメータ，低いと決定論的に振る舞い，高いとその都度振る舞いが異なる
            seed: int
                乱数の種 指定しなければ現在の時刻を用いる。指定すると再現性が保たれる
            verbose: bool
                冗長な出力を行う
            w2v: w2v
                word2vec の本体
        """
        np.random.seed(seed)  # 乱数のタネを設定

        ret = []  # 戻り値
        for i in range(n_seq):
            #vec = self.w2v[word]  # ターゲット語のベクトル

            # 以下の行が 2021年2月23日の議論でのポイント，`self.wProj ` の部分を `self.Proj` に変更すると
            # 直交射影行列への射影ベクトルを得ることができる
            vec = np.dot(self.wProj, self.w2v[word])  # 射影行列をかけて 動物空間へ射影する
            names = self.w2v.similar_by_vector(vec)
            entries = np.array([x[0] for x in names])[1:]  # 最初のエントリーだけ外す [1:] の部分，
                                                           # 保続にするなら  [1:] を削除する
            sims = np.array([x[1] for x in names])[1:]
            probs = np.exp(sims/beta)/np.exp(sims/beta).sum()  # softmax
            word  = np.random.choice(entries,p=probs)
            ret.append(word)
            if verbose:
                print(word, end="")
                print(' :', entries.index(word)+1, entries, probs)
        return ret
        
# 上の確率密度を使ってランダムサンプリングしてみる
#word = np.random.choice(entries,p=P_sim)


In [15]:
f = animal_forest()

targets = ['イヌ', 'サンショウウオ','アニメ']
for target in targets:
    print(target, ':->', f.word_assoc(target, beta=0.05,n_seq=7))
    print('\tオリジナル word2vec による最近隣語', [x[0]for x in w2v.similar_by_vector(w2v[target])])


イヌ :-> ['カワウソ', 'ワニ', 'ザリガニ', 'ザリガニ', 'ウミヘビ', 'カワウソ', 'ニホンアナグマ']
	オリジナル word2vec による最近隣語 ['イヌ', 'ネコ', 'オオカミ', 'ブタ', 'ウサギ', 'ウシ', '犬', 'アライグマ', 'ニワトリ', 'ハツカネズミ']
サンショウウオ :-> ['ザリガニ', 'アリクイ', 'ワニ', 'ヘビ', '猛禽類', 'アカギツネ', 'ザリガニ']
	オリジナル word2vec による最近隣語 ['サンショウウオ', 'タイコウチ', 'カナヘビ', 'アオガエル', 'は虫類', 'イモリ', 'アカハライモリ', 'オオサンショウウオ', 'アマガエル', 'ゲンゴロウ']
アニメ :-> ['ネコ', 'ザリガニ', 'ワニ', 'ヘビ', '猛禽類', 'アカギツネ', 'ザリガニ']
	オリジナル word2vec による最近隣語 ['アニメ', 'テレビアニメ', 'アニメ版', '実写', 'OVA', '本編', '漫画', 'キャラクター', 'アニメーション', 'アニメ化']
